CLEANING DATA ON NEW YORK CITY AIRBNB OPEN DATA 

In [136]:
import pandas as pd  
from scipy.stats import zscore 
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sklearn_preprocessing
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhara\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [137]:
df=pd.read_csv("C:/Users/bhara/Downloads/AB_NYC_2019.csv")

In [138]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [139]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [140]:
df.shape

(48895, 16)

In [141]:
df.isnull().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [142]:
df.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [144]:
df.duplicated().sum()

np.int64(0)

In [145]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


From this section of initial exploration I have observed the following:
   1.name,neighbourhood group,last_review and review_per_month columns have nun values.
   2.There are no duplicate values in the dataset.

In [146]:
# IMPUTATION 

In [147]:
df['name'].mode()

0    Hillside Hotel
Name: name, dtype: object

In [148]:
df['name']=df['name'].fillna("Hillside Hotel")

In [149]:
df.isnull().sum()

id                                    0
name                                  0
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [150]:
df['host_name'].mode()

0    Michael
Name: host_name, dtype: object

In [151]:
df['host_name']=df['host_name'].fillna("Michael")

In [152]:
df['last_review'].mode()

0    2019-06-23
Name: last_review, dtype: object

In [153]:
df['last_review']=df['last_review'].fillna("2019-06-23")

In [154]:
df['reviews_per_month'].mean()

np.float64(1.3732214298586618)

In [155]:
df['reviews_per_month']=df['reviews_per_month'].fillna("1.3732214298586618")

In [156]:
df.isnull().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

1.name,neighbourhood_group,last_review, and reviews_per_month column has several nun values before this section

2.Imputed the nun values in these columns by making use of statistical funtions such as mean and mode

In [157]:
# DATA PARSING

In [158]:
try:
    df['last_review']= pd.to_datetime(df['last_review'], format='Xd/Xm/Xy')
except ValueError:
    df['last_review'] = pd.to_datetime(df['last_review'],errors='coerce')

In [159]:
df['last_review']

0       2018-10-19
1       2019-05-21
2       2019-06-23
3       2019-07-05
4       2018-11-19
           ...    
48890   2019-06-23
48891   2019-06-23
48892   2019-06-23
48893   2019-06-23
48894   2019-06-23
Name: last_review, Length: 48895, dtype: datetime64[ns]

In [160]:
# OUTLIER DETECTION AND HANDLING 

In [161]:
df['z_latitude']=0.0

In [162]:
df['z_latitude']=zscore(df['latitude'])

In [163]:
condition_to_drop = (df['z_latitude'] > 3.0)
df = df[condition_to_drop]
condition_to_drop = (df['z_latitude'] < -3.0)
df = df[condition_to_drop]

In [164]:
df[(df['z_latitude'] < -3.0)]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude


In [165]:
df[(df['z_latitude'] > 3.0)]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude


In [166]:
df['z_longitude']=0.0

In [167]:
df['z_longitude']=zscore(df['longitude'])

In [168]:
df['z_longitude']

Series([], Name: z_longitude, dtype: float64)

In [169]:
df.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude


In [170]:
condition_to_drop = (df['z_longitude'] < -3.0)
df = df[condition_to_drop]
condition_to_drop = (df['z_longitude'] > 3.0)
df = df[condition_to_drop]

In [171]:
df[(df['z_longitude'] < -3.0)]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude


In [172]:
df[(df['z_longitude'] > 3.0)]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude


In [173]:
df['z_price']=0.0

In [174]:
df['z_price']=zscore(df['price'])

In [175]:
df['z_price']

Series([], Name: z_price, dtype: float64)

In [176]:
condition_to_be_checked=(df['z_price']>3.0)

In [177]:
df[(df['z_price']>3.0)]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude,z_price


In [178]:
df[(df['z_price']< -3.0)]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude,z_price


1. Outliers can cause problems in model building so I decided to remove them from the dataset

2. I utilized z score technique to detect outliers in latitude, longitude and price columns as they are the most crucial columns to the analysis.

3 If z score lies between -3 and +3 then the data point is not an outlier otherwise it is classified as an outlier.

4. I created columns of z latitude and z longitude to store the z-scores of latitude and longitude columns respectively.

5. I removed the rows containing z latitude > 3 and z_latitude<-3

6. There were no rows for z longitude<-3.
    
7. I removed the rows where z longitude>3

8. I created columns of z price to store the z-scores of price column.

9. Since most of the outliers detected through the price column have names such as "luxury" and "trendy" and might have a reason for their high price, I will be keeping the outliers in the price columns as it is and not deal with them.

10. There were no rows for z price<-3

In [179]:
# TEXT PRE PROCESSING 

In [180]:
def remove_stopwords(text):
    token_text = word_tokenize(text)
    filter_text= [word for word in token_text if word.lower()not in stopwords.words('english')]
    clean_text =' ',join(filter_text)
    return clean_text

In [181]:
df['name'].head(2)

Series([], Name: name, dtype: object)

In [182]:
df['name']=df['name'].apply(remove_stopwords)

In [183]:
df['name'].head(2)


Series([], Name: name, dtype: object)

In [184]:
punc=string.punctuation

In [185]:
punc

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [186]:
def remove_punctuation(text):
    clean_text=""
    for c in text:
        if c not in punc:
            clean_text=clean_text+""+c 
    return clean_text

        


In [187]:
df['name']=df['name'].apply(remove_punctuation)
df['host_name']=df['host_name'].apply(remove_punctuation)
df['neighbourhood_group']=df['neighbourhood_group'].apply(remove_punctuation)
df['neighbourhood']=df['neighbourhood'].apply(remove_punctuation)

In [188]:
df.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude,z_price


In [189]:
def to_lower_case(text):
    return text.lower()

In [190]:
df['name']=df['name'].apply(to_lower_case)
df['host_name']=df['host_name'].apply(to_lower_case)
df['neighbourhood_group']=df['neighbourhood_group'].apply(to_lower_case)
df['neighbourhood']=df['neighbourhood'].apply(to_lower_case)
df['room_type']=df['room_type'].apply(to_lower_case)

In [191]:
df.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude,z_price


In [192]:
# COLUMN REMOVAL 

In [193]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'z_latitude', 'z_longitude', 'z_price'],
      dtype='object')

In [194]:
df=df.drop(['id'],axis=1)
df=df.drop(['host_name'],axis=1)
df=df.drop(['z_latitude'],axis=1)
df=df.drop(['z_longitude'],axis=1)
df=df.drop(['z_price'],axis=1)

In [195]:
df.columns

Index(['name', 'host_id', 'neighbourhood_group', 'neighbourhood', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

In [196]:
# LABEL ENCODING

In [197]:
df.dtypes

name                                      object
host_id                                    int64
neighbourhood_group                       object
neighbourhood                             object
latitude                                 float64
longitude                                float64
room_type                                 object
price                                      int64
minimum_nights                             int64
number_of_reviews                          int64
last_review                       datetime64[ns]
reviews_per_month                         object
calculated_host_listings_count             int64
availability_365                           int64
dtype: object

In [198]:
from sklearn.preprocessing import LabelEncoder


In [199]:
le=LabelEncoder()

In [200]:
df["neighbourhood_group"]=le.fit_transform(df["neighbourhood_group"])
df["neighbourhood"]=le.fit_transform(df["neighbourhood"])
df["room_type"]=le.fit_transform(df["room_type"])

In [201]:
df.head(2)

,name,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365


In [202]:
df.columns

Index(['name', 'host_id', 'neighbourhood_group', 'neighbourhood', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')